# Introduction

## Preparations

In [81]:
# import required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [82]:
# load data
data = pd.read_csv('../../data/V-Dem-CY-Core-v14.csv')

## Explore the Data

In [83]:
print(data.head())

  country_name country_text_id  country_id  year historical_date  project  \
0       Mexico             MEX           3  1789      1789-12-31        1   
1       Mexico             MEX           3  1790      1790-12-31        1   
2       Mexico             MEX           3  1791      1791-12-31        1   
3       Mexico             MEX           3  1792      1792-12-31        1   
4       Mexico             MEX           3  1793      1793-12-31        1   

   historical                  histname  codingstart  codingend  ...  \
0           1  Viceroyalty of New Spain         1789       2023  ...   
1           1  Viceroyalty of New Spain         1789       2023  ...   
2           1  Viceroyalty of New Spain         1789       2023  ...   
3           1  Viceroyalty of New Spain         1789       2023  ...   
4           1  Viceroyalty of New Spain         1789       2023  ...   

   v2xex_elecleg  v2xps_party  v2xps_party_codelow  v2xps_party_codehigh  \
0            0.0          Na

## Filter data

In [84]:
# Create subset of data with fewer variables
subset_columns = ['country_name', 'year', 'v2x_regime', 'v2x_polyarchy', 'v2x_libdem', 'v2x_civlib', 'v2x_rule', 'v2x_egaldem']
democracy_df = data[subset_columns].copy()
print(democracy_df.head())

  country_name  year  v2x_regime  v2x_polyarchy  v2x_libdem  v2x_civlib  \
0       Mexico  1789         NaN          0.028       0.045       0.221   
1       Mexico  1790         NaN          0.028       0.045       0.221   
2       Mexico  1791         NaN          0.028       0.045       0.221   
3       Mexico  1792         NaN          0.028       0.045       0.221   
4       Mexico  1793         NaN          0.028       0.045       0.221   

   v2x_rule  v2x_egaldem  
0     0.272          NaN  
1     0.272          NaN  
2     0.272          NaN  
3     0.272          NaN  
4     0.272          NaN  


In [85]:
# Filter for observations after 1990
democracy_df_recent = democracy_df[democracy_df['year'] >= 1990].copy()
print(democracy_df_recent.head())

    country_name  year  v2x_regime  v2x_polyarchy  v2x_libdem  v2x_civlib  \
201       Mexico  1990         1.0          0.393       0.191       0.606   
202       Mexico  1991         1.0          0.415       0.202       0.618   
203       Mexico  1992         1.0          0.441       0.216       0.628   
204       Mexico  1993         1.0          0.455       0.221       0.640   
205       Mexico  1994         1.0          0.474       0.261       0.652   

     v2x_rule  v2x_egaldem  
201     0.338        0.193  
202     0.338        0.204  
203     0.338        0.215  
204     0.338        0.220  
205     0.456        0.233  


## Clean the Dataset

In [86]:
# check variable types
print(democracy_df_recent.dtypes)

country_name      object
year               int64
v2x_regime       float64
v2x_polyarchy    float64
v2x_libdem       float64
v2x_civlib       float64
v2x_rule         float64
v2x_egaldem      float64
dtype: object


In [87]:
# adjust variable type to make 'country' variable more efficient (as string rather than object'
democracy_df_recent['country_name'] = democracy_df_recent['country_name'].astype('string')
print(democracy_df_recent.dtypes)

country_name     string[python]
year                      int64
v2x_regime              float64
v2x_polyarchy           float64
v2x_libdem              float64
v2x_civlib              float64
v2x_rule                float64
v2x_egaldem             float64
dtype: object


In [88]:
# transformation of variables
# part I: rename variables - using a list

# create a list with new column names
new_column_names = ['country', 'year', 'regime_type', 'polyarchy', 'liberal_dem', 'civil_liberties', 'rule_of_law', 'egalitarian_dem']

democracy_df_recent.columns = new_column_names


In [89]:
# transform variables: recode from 0-1 to 0.100 scale - using a loop
numeric_columns = democracy_df_recent.select_dtypes(include=['number'])

min_values = numeric_columns.min()
max_values = numeric_columns.max()
print("Min ", min_values, ", Max ", max_values)

Min  year               1990.000
regime_type           0.000
polyarchy             0.013
liberal_dem           0.005
civil_liberties       0.015
rule_of_law           0.009
egalitarian_dem       0.022
dtype: float64 , Max  year               2023.000
regime_type           3.000
polyarchy             0.922
liberal_dem           0.897
civil_liberties       0.976
rule_of_law           0.999
egalitarian_dem       0.886
dtype: float64


In [90]:
transform_variables = ['polyarchy', 'liberal_dem', 'civil_liberties', 'rule_of_law', 'egalitarian_dem']
for variable in transform_variables:
    democracy_df_recent[variable] = democracy_df_recent[variable] * 100
print(democracy_df_recent.head())

    country  year  regime_type  polyarchy  liberal_dem  civil_liberties  \
201  Mexico  1990          1.0       39.3         19.1             60.6   
202  Mexico  1991          1.0       41.5         20.2             61.8   
203  Mexico  1992          1.0       44.1         21.6             62.8   
204  Mexico  1993          1.0       45.5         22.1             64.0   
205  Mexico  1994          1.0       47.4         26.1             65.2   

     rule_of_law  egalitarian_dem  
201         33.8             19.3  
202         33.8             20.4  
203         33.8             21.5  
204         33.8             22.0  
205         45.6             23.3  


In [91]:
# check for NA values
print(democracy_df_recent.isna().sum())

country             0
year                0
regime_type         2
polyarchy           2
liberal_dem        23
civil_liberties     0
rule_of_law         0
egalitarian_dem     2
dtype: int64
